In [1]:
from Fairness_aware_model import FairnessAwareModel
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

In [2]:
dataCar = pd.read_csv('dataCar_clean.csv') # contient le train-test split pour comparer avec R au besoin
#clm = dataCar['clm']
#Xtrain, test, _, _ = train_test_split(dataCar, clm, test_size=0.2)
train=dataCar.loc[dataCar['train']==1]
test=dataCar.loc[dataCar['train']==0]

# Modèle baseline avec la méthode bfgs

In [3]:
mod = smf.glm(formula="clm ~ veh_value + veh_body + C(veh_age) + area + gender + C(agecat)", family=sm.families.Binomial(), data=train)
res = mod.fit(method="bfgs")
print(res.summary()) # donne la même chose qu'en R

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    clm   No. Observations:                54285
Model:                            GLM   Df Residuals:                    54257
Model Family:                Binomial   Df Model:                           27
Link Function:                  Logit   Scale:                          1.0000
Method:                          bfgs   Log-Likelihood:                -13435.
Date:                Sun, 13 Nov 2022   Deviance:                       26870.
Time:                        14:00:53   Pearson chi2:                 5.43e+04
No. Iterations:                     0   Pseudo R-squ. (CS):           0.002612
Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.9774      0.48

In [4]:
protected_attributes = ["gender", "agecat"]
# Standard scaling for regression
binary_answer = dataCar["clm"].values
numclaim = dataCar["numclaims"].values
reg_claim = dataCar["claimcst0"].values

In [5]:
"gender" in ["A", "b", "gender", "c"]

True

In [6]:
def hot_encoder(data, optional_columns):
    categorical_cols = []
    for column in data.columns:
            if data[column].dtype == object or column in optional_columns:
                categorical_cols.append(column)

    to_return = pd.get_dummies(data, columns = categorical_cols)
    return to_return

dataCar_encoded = hot_encoder(dataCar, ["gender", "agecat"])

dataCar_encoded = dataCar_encoded.drop(["clm", "numclaims", "claimcst0", "veh_body_BUS","area_A","exposure", "gender_F", "agecat_1"], axis=1)

In [7]:
train_encoded = dataCar_encoded.loc[dataCar_encoded['train']==1]
test_encoded = dataCar_encoded.loc[dataCar_encoded['train']==0]
y = dataCar['clm']

y_train = dataCar.loc[dataCar['train']==1, 'clm']
y_test = dataCar.loc[dataCar['train']==0, 'clm']
X_train_encoded = train_encoded.drop(["train"], axis=1)
X_test_encoded = test_encoded.drop(["train"], axis=1)

In [8]:
# question : à quoi ça sert de faire reshape?
beta = np.ones(X_train_encoded.shape[1])
predicted_x = X_train_encoded @ beta
print(np.sum(y_train.values.reshape(-1,1) * predicted_x.values.reshape(-1,1) - np.exp(predicted_x.values.reshape(-1,1))))
print(np.sum(y_train * predicted_x - np.exp(predicted_x)))

-5.5771202852141544e+16
-5.5771202852141544e+16


In [9]:
X_train_encoded.columns.values

array(['veh_value', 'veh_age', 'veh_body_CONVT', 'veh_body_COUPE',
       'veh_body_HBACK', 'veh_body_HDTOP', 'veh_body_MCARA',
       'veh_body_MIBUS', 'veh_body_PANVN', 'veh_body_RDSTR',
       'veh_body_SEDAN', 'veh_body_STNWG', 'veh_body_TRUCK',
       'veh_body_UTE', 'gender_M', 'area_B', 'area_C', 'area_D', 'area_E',
       'area_F', 'agecat_2', 'agecat_3', 'agecat_4', 'agecat_5',
       'agecat_6'], dtype=object)

In [10]:
model2 = sm.Logit(y_train, sm.add_constant(X_train_encoded)).fit()
print(model2.summary())

Optimization terminated successfully.
         Current function value: 0.247637
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                    clm   No. Observations:                54285
Model:                          Logit   Df Residuals:                    54259
Method:                           MLE   Df Model:                           25
Date:                Sun, 13 Nov 2022   Pseudo R-squ.:                0.004681
Time:                        14:00:53   Log-Likelihood:                -13443.
converged:                       True   LL-Null:                       -13506.
Covariance Type:            nonrobust   LLR p-value:                 1.596e-15
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -1.8913      0.489     -3.865      0.000      -2.850      -0.932
veh_value        

In [11]:
protected_attributes = [np.where(dataCar.columns.values == i)[0][0] for i in protected_attributes] # position des colonnes d'attributs protégés dans les données

array([[1.  , 3.26, 2.  , ..., 0.  , 0.  , 0.  ],
       [1.  , 0.72, 4.  , ..., 0.  , 0.  , 0.  ],
       [1.  , 2.01, 3.  , ..., 1.  , 0.  , 0.  ],
       ...,
       [1.  , 1.23, 4.  , ..., 0.  , 0.  , 0.  ],
       [1.  , 0.97, 3.  , ..., 0.  , 0.  , 0.  ],
       [1.  , 1.02, 3.  , ..., 0.  , 0.  , 0.  ]])

In [28]:
fam = FairnessAwareModel(regularization=0, protected_attributes=protected_attributes, family="binomial", offset=1)
fam.fit(np.column_stack([np.ones(X_train_encoded.shape[0]), np.array(X_train_encoded)]), y_train.values) # il faut inclute un "intercept", donc colonne de 1

[0.03846154 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154
 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154
 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154
 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154
 0.03846154 0.03846154]
calcul vraisemblance logistique
[0.03846155 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154
 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154
 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154
 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154
 0.03846154 0.03846154]
calcul vraisemblance logistique
[0.03846154 0.03846155 0.03846154 0.03846154 0.03846154 0.03846154
 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154
 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154
 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154 0.03846154
 0.03846154 0.03846154]
calcul vraisemblance logistique
[0.03846154 0.03846154 0.038

/media/simon/Elements/Universite/Maitrise statistique/5-A22/Actulab-coop/Fairness_aware_model.py:64: RuntimeWarning: divide by zero encountered in log
  return np.sum(y.reshape(-1,1) * (np.log(pi) - np.log(1-pi)) + np.log(1-pi) )
/media/simon/Elements/Universite/Maitrise statistique/5-A22/Actulab-coop/Fairness_aware_model.py:64: RuntimeWarning: invalid value encountered in multiply
  return np.sum(y.reshape(-1,1) * (np.log(pi) - np.log(1-pi)) + np.log(1-pi) )
/media/simon/Elements/Universite/Maitrise statistique/5-A22/Actulab-coop/Fairness_aware_model.py:64: RuntimeWarning: invalid value encountered in add
  return np.sum(y.reshape(-1,1) * (np.log(pi) - np.log(1-pi)) + np.log(1-pi) )


[9.87918830e+01 1.75785459e+02 2.65328449e+02 1.82857777e-01
 1.13057640e+00 2.71735082e+01 2.35242294e+00 2.14884499e-01
 1.08044867e+00 1.10145492e+00 8.43153469e-02 3.24594729e+01
 2.38196101e+01 2.59508129e+00 6.94994380e+00 4.33916369e+01
 1.93524196e+01 3.00834705e+01 1.22301638e+01 8.72937264e+00
 5.17832740e+00 1.86183140e+01 2.30056791e+01 2.34955344e+01
 1.60500110e+01 9.98845593e+00]
calcul vraisemblance logistique
[9.87918830e+01 1.75785459e+02 2.65328449e+02 1.82857777e-01
 1.13057640e+00 2.71735082e+01 2.35242294e+00 2.14884499e-01
 1.08044867e+00 1.10145491e+00 8.43153618e-02 3.24594729e+01
 2.38196101e+01 2.59508129e+00 6.94994380e+00 4.33916369e+01
 1.93524196e+01 3.00834705e+01 1.22301638e+01 8.72937264e+00
 5.17832740e+00 1.86183140e+01 2.30056791e+01 2.34955344e+01
 1.60500110e+01 9.98845593e+00]
calcul vraisemblance logistique
[9.87918830e+01 1.75785459e+02 2.65328449e+02 1.82857777e-01
 1.13057640e+00 2.71735082e+01 2.35242294e+00 2.14884499e-01
 1.08044867e+00 1.

/media/simon/Elements/Universite/Maitrise statistique/5-A22/Actulab-coop/Fairness_aware_model.py:64: RuntimeWarning: divide by zero encountered in log
  return np.sum(y.reshape(-1,1) * (np.log(pi) - np.log(1-pi)) + np.log(1-pi) )
/media/simon/Elements/Universite/Maitrise statistique/5-A22/Actulab-coop/Fairness_aware_model.py:64: RuntimeWarning: invalid value encountered in multiply
  return np.sum(y.reshape(-1,1) * (np.log(pi) - np.log(1-pi)) + np.log(1-pi) )
/media/simon/Elements/Universite/Maitrise statistique/5-A22/Actulab-coop/Fairness_aware_model.py:64: RuntimeWarning: invalid value encountered in add
  return np.sum(y.reshape(-1,1) * (np.log(pi) - np.log(1-pi)) + np.log(1-pi) )


[1.85728280e+01 3.30232353e+01 4.98289578e+01 6.55622987e-02
 2.43433243e-01 5.13125622e+00 4.72753414e-01 7.15731792e-02
 2.34025106e-01 2.37967625e-01 4.70675318e-02 6.12334345e+00
 4.50178561e+00 5.18296331e-01 1.33563123e+00 8.17512788e+00
 3.66336864e+00 5.67740750e+00 2.32664027e+00 1.66960028e+00
 1.00312845e+00 3.52558927e+00 4.34902436e+00 4.44096202e+00
 3.04356174e+00 1.90590917e+00]
calcul vraisemblance logistique
[1.85728280e+01 3.30232353e+01 4.98289578e+01 6.55622987e-02
 2.43433243e-01 5.13125622e+00 4.72753414e-01 7.15731792e-02
 2.34025106e-01 2.37967625e-01 4.70675318e-02 6.12334345e+00
 4.50178561e+00 5.18296331e-01 1.33563123e+00 8.17512788e+00
 3.66336863e+00 5.67740751e+00 2.32664027e+00 1.66960028e+00
 1.00312845e+00 3.52558927e+00 4.34902436e+00 4.44096202e+00
 3.04356174e+00 1.90590917e+00]
calcul vraisemblance logistique
[1.85728280e+01 3.30232353e+01 4.98289578e+01 6.55622987e-02
 2.43433243e-01 5.13125622e+00 4.72753414e-01 7.15731792e-02
 2.34025106e-01 2.

/media/simon/Elements/Universite/Maitrise statistique/5-A22/Actulab-coop/Fairness_aware_model.py:64: RuntimeWarning: divide by zero encountered in log
  return np.sum(y.reshape(-1,1) * (np.log(pi) - np.log(1-pi)) + np.log(1-pi) )
/media/simon/Elements/Universite/Maitrise statistique/5-A22/Actulab-coop/Fairness_aware_model.py:64: RuntimeWarning: invalid value encountered in multiply
  return np.sum(y.reshape(-1,1) * (np.log(pi) - np.log(1-pi)) + np.log(1-pi) )
/media/simon/Elements/Universite/Maitrise statistique/5-A22/Actulab-coop/Fairness_aware_model.py:64: RuntimeWarning: invalid value encountered in add
  return np.sum(y.reshape(-1,1) * (np.log(pi) - np.log(1-pi)) + np.log(1-pi) )


In [38]:
pd.DataFrame({"variable":["intercept"] + X_train_encoded.columns.tolist(), "beta":fam.beta})

,variable,beta
0,intercept,296.588325
1,veh_value,527.794842
2,veh_age,796.686401
3,veh_body_CONVT,0.472074
4,veh_body_COUPE,3.318009
5,veh_body_HBACK,81.523176
6,veh_body_HDTOP,6.987132
7,veh_body_MCARA,0.568248
8,veh_body_MIBUS,3.167479
9,veh_body_PANVN,3.230559
